In [1]:
import numpy as np
import imageio.v2 as iio
from PIL import Image
import scipy.ndimage as sn
from skimage import filters
import random
import colorsys

img = iio.imread('a.jpg')
img_np = np.array(img)

mean_filtered = sn.uniform_filter(img_np, size=(5, 5, 1))
iio.imwrite('a_mean.jpg', mean_filtered.astype(np.uint8))

gray = np.mean(img_np, axis=2).astype(np.uint8)
edges = filters.sobel(gray) 
edges = (edges * 255).astype(np.uint8)
iio.imwrite('a_edge.jpg', edges)

perm = np.random.permutation(3)
shuffled_img = img_np[:, :, perm]
iio.imwrite('a_random_color.jpg', shuffled_img)

def rgb_to_hsv_image(img):
    img = img / 255.0
    hsv = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            hsv[i, j] = colorsys.rgb_to_hsv(*img[i, j])
    return hsv

hsv_img = rgb_to_hsv_image(img_np.astype(float))
h = (hsv_img[:, :, 0] * 255).astype(np.uint8)
s = (hsv_img[:, :, 1] * 255).astype(np.uint8)
v = (hsv_img[:, :, 2] * 255).astype(np.uint8)

iio.imwrite('a_hue.jpg', h)
iio.imwrite('a_saturation.jpg', s)
iio.imwrite('a_value.jpg', v)

In [2]:
import cv2
import numpy as np
import random
import os

image_files = ['image1.jpg', 'image2.jpg', 'image3.jpg']

# Hàm đọc tất cả ảnh
def read_images():
    images = []
    for f in image_files:
        img = cv2.imread(f, cv2.IMREAD_GRAYSCALE)  # Dùng ảnh grayscale cho xử lý histogram
        if img is not None:
            images.append(img)
        else:
            print(f"Lỗi: Không thể đọc file {f}")
    return images

# 1. Image Inverse (đảo ảnh)
def inverse_image(img):
    return 255 - img

# 2. Gamma Correction
def gamma_correction(img, gamma=None):
    if gamma is None:
        gamma = random.uniform(0.5, 2.0)
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, table)

# 3. Log Transformation
def log_transformation(img, c=None):
    if c is None:
        c = random.uniform(1.0, 5.0)
    img_float = img.astype(np.float32)
    log_img = c * np.log1p(img_float)
    log_img = np.clip((log_img / log_img.max()) * 255, 0, 255).astype(np.uint8)
    return log_img

# 4. Histogram Equalization
def histogram_equalization(img):
    return cv2.equalizeHist(img)

# 5. Contrast Stretching
def contrast_stretch(img):
    min_val = random.randint(0, 50)
    max_val = random.randint(200, 255)
    stretched = np.clip((img - min_val) * 255.0 / (max_val - min_val), 0, 255)
    return stretched.astype(np.uint8)

# 6. Adaptive Histogram Equalization (CLAHE)
def adaptive_hist_eq(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(img)

# Map phím sang hàm xử lý
transformations = {
    'I': ('inverse', inverse_image),
    'G': ('gamma', gamma_correction),
    'L': ('log', log_transformation),
    'H': ('histogram', histogram_equalization),
    'C': ('contrast', contrast_stretch),
    'A': ('adaptive', adaptive_hist_eq)
}

# Hiển thị menu lựa chọn
print("=== MENU BIẾN ĐỔI ẢNH ===")
print("Nhấn phím I: Image inverse transformation")
print("Nhấn phím G: Gamma Correction")
print("Nhấn phím L: Log Transformation")
print("Nhấn phím H: Histogram Equalization")
print("Nhấn phím C: Contrast Stretching")
print("Nhấn phím A: Adaptive Histogram Equalization")
choice = input("Vui lòng nhập phím tương ứng: ").upper()

# Kiểm tra phím nhập
if choice in transformations:
    method_name, func = transformations[choice]
    images = read_images()

    for idx, img in enumerate(images, start=1):
        result = func(img)  # Áp dụng biến đổi ảnh
        filename = f"output_{method_name}_{idx}.jpg"
        cv2.imwrite(filename, result)
        print(f"Đã lưu ảnh: {filename}")
else:
    print("Phím không hợp lệ.")

=== MENU BIẾN ĐỔI ẢNH ===
Nhấn phím I: Image inverse transformation
Nhấn phím G: Gamma Correction
Nhấn phím L: Log Transformation
Nhấn phím H: Histogram Equalization
Nhấn phím C: Contrast Stretching
Nhấn phím A: Adaptive Histogram Equalization
Đã lưu ảnh: output_log_1.jpg
Đã lưu ảnh: output_log_2.jpg
Đã lưu ảnh: output_log_3.jpg


In [3]:
import cv2
import numpy as np
import random
import os

def read_image_safely(filename):
    img = cv2.imread(filename)
    if img is None:
        print(f"Không thể đọc file: {filename}. Hãy kiểm tra lại tên hoặc đường dẫn.")
    return img

img_fruits = read_image_safely('colorful-ripe-tropical-fruits.jpg')
if img_fruits is not None:
    img_fruits_resized = cv2.copyMakeBorder(img_fruits, 30, 30, 30, 30, borderType=cv2.BORDER_REPLICATE)
    cv2.imwrite('output_fruits_resized.jpg', img_fruits_resized)
    print("Đã xử lý ảnh trái cây (resize + padding).")

def rotate_and_flip(img, angle):
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    flipped = cv2.flip(rotated, 1)
    return flipped

img_quangninh = read_image_safely('quang_ninh.jpg')
if img_quangninh is not None:
    img_quangninh_transformed = rotate_and_flip(img_quangninh, 45)
    cv2.imwrite('output_quangninh_rotated_flipped.jpg', img_quangninh_transformed)
    print("Đã xử lý ảnh Quảng Ninh (xoay + lật).")

def upscale_and_blur(img, scale=5, kernel_size=(7,7)):
    upscaled = cv2.resize(img, (0,0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    blurred = cv2.GaussianBlur(upscaled, kernel_size, 0)
    return blurred

img_pagoda = read_image_safely('pagoda.jpg')
if img_pagoda is not None:
    img_pagoda_blurred = upscale_and_blur(img_pagoda)
    cv2.imwrite('output_pagoda_upscaled_blur.jpg', img_pagoda_blurred)
    print("Đã xử lý ảnh Pagoda (resize + blur).")

    def adjust_brightness_contrast(img, alpha=None, beta=None):
        if alpha is None:
            alpha = random.uniform(0.5, 2.0)
        if beta is None:
            beta = random.randint(-50, 50)
        print(f"Áp dụng alpha = {alpha:.2f}, beta = {beta}")
        adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
        return adjusted

    img_pagoda_adjusted = adjust_brightness_contrast(img_pagoda)
    cv2.imwrite('output_pagoda_contrast_brightness.jpg', img_pagoda_adjusted)
    print("Đã áp dụng contrast/brightness lên ảnh Pagoda.")

Đã xử lý ảnh trái cây (resize + padding).
Đã xử lý ảnh Quảng Ninh (xoay + lật).
Đã xử lý ảnh Pagoda (resize + blur).
Áp dụng alpha = 1.48, beta = 43
Đã áp dụng contrast/brightness lên ảnh Pagoda.
